In [1]:
from FINDER import FINDER
import os
import numpy as np
import pandas as pd
import tsplib95
import networkx as nx
import re


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
config_path = 'best_models/tsp_2d/nrange_10_20_len_101341/config.txt'
dqn = FINDER(config_path=config_path)

{'help_func': 1, 'max_bp_iter': 3, 'aggregatorID': 0, 'node_init_dim': 4, 'edge_init_dim': 4, 'state_init_dim': 4, 'node_embed_dim': 64, 'edge_embed_dim': 64, 'edge_embed': 2, 'ignore_covered_edges': 0, 'selected_nodes_inclusion': 2, 'IsHuberloss': 0, 'BATCH_SIZE': 64, 'initialization_stddev': 0.01, 'MAX_ITERATION': 150000, 'LEARNING_RATE': 0.0001, 'Alpha': 0.001, 'save_interval': 300, 'num_env': 1, 'g_type': 'tsp_2d', 'NUM_MIN': 10, 'NUM_MAX': 20, 'n_generator': 1000, 'decoder': 0, 'REG_HIDDEN': 32, 'IsDoubleDQN': 0, 'N_STEP': 5, 'GAMMA': 1.0, 'UPDATE_TIME': 1000, 'eps_start': 1.0, 'eps_end': 0.05, 'eps_step': 10000.0, 'MEMORY_SIZE': 150000, 'valid_path': 'valid_sets/synthetic_nrange_15_20_200/', 'valid_scale_fac': 0.0001, 'n_valid': 200, 'IsPrioritizedSampling': 0, 'epsilon': 1e-07, 'alpha': 0.6, 'beta': 0.4, 'beta_increment_per_sampling': 0.001, 'TD_err_upper': 1.0}
Sucessfully loaded key 'help_func' from external config file!
Sucessfully loaded key 'max_bp_iter' from external confi

In [3]:
# data_dir = 'test_sets/synthetic_nrange_10_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_15_20_1000/'
data_dir = 'test_sets/synthetic_nrange_40_50_1000/'
# data_dir = 'valid_sets/synthetic_nrange_10_20_200/'
# data_dir = 'valid_sets/synthetic_nrange_15_20_200/'

In [4]:
graph_list = []

atoi = lambda text : int(text) if text.isdigit() else text
natural_keys = lambda text : [atoi(c) for c in re.split('(\d+)', text)]

fnames = os.listdir(data_dir)
fnames.sort(key=natural_keys)

for fname in fnames:
    try:
        if not 'tsp' in fname:
            continue
        problem = tsplib95.load(data_dir + fname)
        graph_list.append(problem.get_graph())
        
    except:
        print('Error, while loading file {}'.format(fname))
print(len(graph_list))

1000


In [5]:
# remove edges from one node to itself
for g in graph_list:
    ebunch=[(k,k) for k in range(len(g.nodes))]
    g.remove_edges_from(ebunch)

In [6]:
# scale size of the graphs such that it fits into 0,1 square
scale_factor = 1/10000
for g in graph_list:
    for node in range(len(g.nodes)):
        g.nodes[node]['coord'] = np.array(g.nodes[node]['coord']) * scale_factor
    for edge in g.edges:
        g.edges[edge]['weight'] = g.edges[edge]['weight'] * scale_factor

In [7]:
num_min = dqn.cfg['NUM_MIN']
num_max = dqn.cfg['NUM_MAX']

base_path = 'best_models/{}/'.format(dqn.cfg['g_type'])
best_tour_length = np.inf
# file_endings = ['ckpt', 'csv', 'pyx']
models = os.scandir(base_path)
for model in models:
    # res = [ele for ele in file_endings if(ele in model)]
    # check whether we have file or folder, continue in case of file
    if model.is_file():
        continue
    new_base_path = base_path + model.name + '/'
    for f in os.listdir(new_base_path):
        # print(f)
        nrange_str = 'nrange_{}_{}'.format(num_min, num_max)
        if ('ckpt' not in f):# or (nrange_str not in f):
            continue
        # print(f)
        f_len = f.split('_')[-1].split('.')[0]
        tour_length = float(f_len)/(10**(len(f_len)-1))
        if f_len[0] != '1':
            continue
            # norm_tour_length = tour_length/float(config['valid_sol'])
        else:
            norm_tour_length = tour_length
        if norm_tour_length < best_tour_length:
            best_model_file = '.'.join(f.split('.')[0:-1])
            best_model_base_path = new_base_path
            best_tour_length = tour_length
print(best_model_file)

nrange_10_20_iter_0_len_101341.ckpt


In [8]:
print(best_model_base_path)

best_models/tsp_2d/nrange_10_20_len_101341/


In [9]:
# load model into Finder
g_type = dqn.cfg['g_type']
best_model = dqn.LoadModel(model_path=best_model_base_path+best_model_file)
lengths = []
solutions = []
k = 0
for g in graph_list:
    len, sol, time = dqn.Evaluate(g)
    lengths.append(len)
    solutions.append(sol)
    print(k)
    k += 1


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from best_models/tsp_2d/nrange_10_20_len_101341/nrange_10_20_iter_0_len_101341.ckpt
model sucessfully restored from file
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
2

In [10]:
true_lengths = None
with open(data_dir+'lengths.txt', 'r') as f:
    lines = f.readlines()
    lines = [float(line.split(':')[-1].strip()) for line in lines]
true_lengths = lines

print(solutions[0])
print(lengths[0])
np.mean([length[0]/length[1] for length in zip(lengths, true_lengths)])

[0, 23, 17, 35, 16, 2, 26, 22, 10, 29, 8, 30, 9, 7, 14, 21, 3, 4, 37, 6, 36, 32, 1, 18, 12, 24, 20, 5, 11, 25, 19, 15, 39, 27, 38, 34, 31, 28, 13, 33]
6.3450000000000015


1.1382610922374465

In [16]:
num_min = 40
num_max = 50
index = pd.RangeIndex(start=num_min, stop=num_max)
sol_df = pd.DataFrame()
idx = 0
for fname in fnames:
    if not 'tsp' in fname:
        continue
    tmp_df = pd.DataFrame()
    tmp_df[fname.split('.')[0]] = solutions[idx]
    sol_df = pd.concat([sol_df,tmp_df.astype(int)], ignore_index=False, axis=1)
    idx += 1
    

In [18]:
best_model = best_model_file.split('.')[0]
sol_df.to_csv('results/{}/solution_nrange_{}_{}_{}.csv'.format(data_dir.split("/")[-1], num_min, num_max, best_model))

In [19]:
lens_df = pd.DataFrame()
idx = 0
for fname in fnames:
    if not 'tsp' in fname:
        continue
    tmp_df = pd.DataFrame()
    tmp_df[fname.split('.')[0]] = [lengths[idx]]
    lens_df = pd.concat([lens_df,tmp_df], ignore_index=False, axis=1)
    idx += 1

In [20]:
lens_df.to_csv('results/{}/tour_lengths_nrange_{}_{}_{}.csv'.format(data_dir.split("/")[-1], num_min, num_max, best_model))

In [14]:
graph = graph_list[0]
length, solution, time_ = dqn.Evaluate(g=graph)

In [15]:
solution, length

([0, 6, 1, 11, 8, 9, 2, 7, 5, 10, 4, 3], 2.7711)

In [11]:
# check whether length calculation is correct
def calc_tour_length(graph, solution):
    tot_len = 0
    for i in range(np.array(solution).shape[0]):
        if i == np.array(solution).shape[0] - 1:
            tot_len += graph[solution[i]][solution[0]]['weight']
        else:
            tot_len += graph[solution[i]][solution[i + 1]]['weight']
    return tot_len

In [12]:
def add_node(graph, cur_sol, new_node):
    cur_dist = 10000000.0
    for i in range(0, np.array(cur_sol).shape[0]):
        if i + 1 == np.array(cur_sol).shape[0]:
            adj = cur_sol[0]
        else:
            adj = cur_sol[i + 1]
        if cur_sol[i] == adj:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight']
        else:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight'] - graph[cur_sol[i]][adj]['weight']
        if cost < cur_dist:
            cur_dist = cost
            pos = i + 1
    cur_sol.insert(pos, new_node)
    return  cur_dist

In [10]:
def calc_helper_tour(g):
    cur_sol = [0]
    rem_nodes = list(np.arange(g.number_of_nodes()))
    rem_nodes.remove(0)
    tour_length = 0
    while np.array(cur_sol).shape[0] < g.number_of_nodes():
        if np.array(rem_nodes).shape[0]-1 == 0:
            rand_idx = 0
        else:
            rand_idx = np.random.randint(0, np.array(rem_nodes).shape[0]-1)
        new_node = rem_nodes[rand_idx]
        tour_length += add_node(g, cur_sol, new_node)
        rem_nodes.remove(new_node)
    return tour_length, cur_sol


In [16]:
max_result = -np.inf
min_result = np.inf
for i in range(100):
    help_lengths = []
    for graph in graph_list:
        help_lengths.append(calc_helper_tour(graph)[0])

    true_lengths = None
    with open(data_dir+'lengths.txt', 'r') as f:
        lines = f.readlines()
        lines = [float(line.split(':')[-1].strip()) for line in lines]
    true_lengths = lines
    result = np.mean([length[0]/length[1] for length in zip(help_lengths, true_lengths)])
    if result < min_result:
        min_result = result
    if result > max_result:
        max_result = result

In [18]:
print(min_result)
print(max_result)

1.0291948212594315
1.0353626379176066


5.9577